# Smadex Technical Task

In [1]:
import pandas as pd
import plotly.express as px

## Data Exploration

In [2]:
df = pd.read_csv("smadex-data.csv", sep=",")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   user_id                     40000 non-null  object 
 1   game_id                     40000 non-null  object 
 2   installers                  40000 non-null  int64  
 3   retained_users              40000 non-null  int64  
 4   minuted_played_install_day  40000 non-null  int64  
 5   Unnamed: 5                  0 non-null      float64
 6   Unnamed: 6                  0 non-null      float64
 7   Unnamed: 7                  0 non-null      float64
 8   Unnamed: 8                  0 non-null      float64
dtypes: float64(4), int64(3), object(2)
memory usage: 2.7+ MB


In [4]:
# drop empty “Unnamed” columns
df.dropna(axis=1, how="all", inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 5 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   user_id                     40000 non-null  object
 1   game_id                     40000 non-null  object
 2   installers                  40000 non-null  int64 
 3   retained_users              40000 non-null  int64 
 4   minuted_played_install_day  40000 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 1.5+ MB


In [6]:
df.head()

,user_id,game_id,installers,retained_users,minuted_played_install_day
0,ZZZyrdne,game_3,1,0,5
1,zzzrsbtQ,game_2,1,0,7
2,zzZRIqJI,game_1,1,1,9
3,zzzPqGoZ,game_4,1,0,7
4,ZZyxJppn,game_1,1,0,1


In [7]:
print(df.columns.tolist())
print(df.shape)

['user_id', 'game_id', 'installers', 'retained_users', 'minuted_played_install_day']
(40000, 5)


In [8]:
df.describe()

,installers,retained_users,minuted_played_install_day
count,40000.0,40000.000000,40000.000000
mean,1.0,0.148250,5.500000
std,0.0,0.355352,2.872317
min,1.0,0.000000,1.000000
25%,1.0,0.000000,3.000000
50%,1.0,0.000000,5.500000
75%,1.0,0.000000,8.000000
max,1.0,1.000000,10.000000


In [9]:
# check for duplicates
df.duplicated().sum()

np.int64(0)

In [10]:
# unique counts
df["game_id"].nunique(), df["user_id"].nunique()

(4, 40000)

In [11]:
# check for missing values
df.isna().sum()

,0
user_id,0
game_id,0
installers,0
retained_users,0
minuted_played_install_day,0


In [12]:
df["minuted_played_install_day"].describe()

,minuted_played_install_day
count,40000.000000
mean,5.500000
std,2.872317
min,1.000000
25%,3.000000
50%,5.500000
75%,8.000000
max,10.000000


In [13]:
# check to ensure there are more installers than retained users
bad = df[df["retained_users"] > df["installers"]]
print(bad.shape)

(0, 5)


In [14]:
# calculate mean and median minutes played per game
time_stats = df.groupby("game_id")["minuted_played_install_day"].agg(["mean", "median"]).reset_index()

# rename columns
time_stats.rename(columns={"mean": "mean minutes played", "median": "median minutes played"}, inplace=True)

print(time_stats)

  game_id  mean minutes played  median minutes played
0  game_1                  5.5                    5.5
1  game_2                  5.5                    5.5
2  game_3                  5.5                    5.5
3  game_4                  5.5                    5.5


## Explore Metrics

### Mean Retention Rate per Game - What percentage of users come back to the app after a period of time?

In [17]:
# compute retention
df["retention_percentage"] = df["retained_users"] / df["installers"] * 100

# group and plot retention metric per game using a bar chart
summary = (
    df
    .groupby("game_id")
    .retention_percentage
    .mean()
    .reset_index()
)

summary["Game ID"] = summary["game_id"].str.replace("game_", "Game ").str.title()

fig = px.bar(
    summary,
    x="Game ID",
    y="retention_percentage",
    title="Retention Rate by Game",
    labels={
        "Game ID": "Game ID",
        "retention_percentage": "Retention (%)"
    }
)

fig.show()

Game 1 performed best based on the percentage of users who came back to the app. Over 1 in 4 players came back signalling excellent initial engagment for the game. It is a clear leader and prime candidate for further investment.

## Retention Curve - Measuring engagement potential

In [16]:
# build retention based on minutes played
bucketed = (
    df
    .groupby(["game_id","minuted_played_install_day"])
    .agg(
        bucket_installs=("installers","sum"),
        bucket_retained=("retained_users","sum")
    )
    .reset_index()
)
bucketed["bucket_retention"] = (
    bucketed["bucket_retained"] / bucketed["bucket_installs"] * 100
)

# improve labels for stakeholder readibility
bucketed["Game ID"] = (
    bucketed["game_id"]
      .str.replace("game_", "Game ")
      .str.title()
)

# plot
fig = px.line(
    bucketed,
    x="minuted_played_install_day",
    y="bucket_retention",
    color="Game ID",
    title="Retention vs Playtime per Game",
    labels={
        "minuted_played_install_day": "Minutes Played",
        "bucket_retention":             "Retention (%)"
    }
)
fig.update_layout(legend_title_text="Game ID")
fig.show()